**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from class_utils import error_histogram
from class_utils.pytorch_utils import EarlyStopping
import torch.nn as nn
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/3jnf3000vwaxtcg/boston_housing.zip?dl=1", directory="data/boston_housing")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Regresný model pre ceny nehnuteľností

V tomto notebook-u budeme aplikovať regresiu na báze umelých neurónových sietí na problém predikcie ceny nehnuteľností. Pracovať budeme s dátovou množinou [Boston housing dataset](https://www.kaggle.com/c/boston-housing).

**Note:**  The example is purely illustrational. The dataset is well-structured (the data is divided into columns with clear meanings etc.), and would therefore probably be approached with a different method in practice – possibly with some approached based on decision trees. Artificial neural networks and deep learning are usually applied to problems with unstructured data, such as images, audio, text etc.

### Načítanie predspracovanie dátovej množiny

Začnime tým, že si zobrazíme opis dát:



In [ ]:
with open("data/boston_housing/description.txt", "r") as file:
    print("".join(file.readlines()))

Ďalej si z CSV súboru načítajme samotnú dátovú množinu:



In [ ]:
df = pd.read_csv("data/boston_housing/housing.csv")
df.head()

#### Rozdelenie dátovej množiny

Ďalej pokračujeme rozdelením dátovej množiny. Dáta budeme v tomto prípade deliť nie na dve, ale až na tri časti: na tréningové, validačné a testovacie dáta v pomere 70 : 5 : 25. Validačná dáta budeme používať počas učenia na regularizáciu a výber modelu (detaily nižšie).

Zároveň pri delení použijeme stratifikáciu podľa diskretizovanej verzie výstupného stĺpca:



In [ ]:
kbins = KBinsDiscretizer(10, encode='ordinal')

y_stratify = kbins.fit_transform(df[["medv"]])
df_train_valid, df_test = train_test_split(df, test_size=0.25,
                                     stratify=y_stratify,
                                     random_state=9)

y_stratify = kbins.fit_transform(df_train_valid[["medv"]])
df_train, df_valid = train_test_split(df_train_valid, test_size=0.05/0.75,
                                     stratify=y_stratify,
                                     random_state=9)

---
#### Úloha 1: Predspracovanie dát

**Aplikujte na dátovú množinu náš štandardný postup predspracovanie pre neurónové siete. Výstupom nech je tréningová množina `X_train`, `Y_train`, validačná množina `X_valid`, `Y_valid` a testovacia množina `X_test`, `Y_test` v príslušnom tvare a s príslušnými dátovými typmi.** 

Pamätajte na to, že `fit_transform` treba použiť len na trénovacej množine. Na validačnú a testovaciu množinu sa používa `transform`.

Nezabudnite dáta konvertovať na PyTorch tenzory s vhodnými dátovými typmi. Tenzory preneste na `device`.

---


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

categorical_inputs = [          ] # -----

numeric_inputs = [              ] # -----

output = ["medv"]


input_preproc = # ---



# -----


output_preproc = StandardScaler()


# -----



---
### Úloha 2: Vytvorenie a tréning neurónovej siete

**Vytvorte neurónovú sieť na regresiu a natrénujte ju na tréningovej množine. Výstupom by mal byť natrénovaný objekt `net` so `scikit-learn` rozhraním, ktorý sa následne bude dať otestovať na testovacích dátach.** 

**Pomôcka: Veľkosti lineárnych vrstiev môžete voliť napr. takéto:** 

* `num_inputs`;
* 128;
* 64;
* 32;
* `num_outputs`;
---


In [ ]:
class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()


        # -----
        


In [ ]:
num_inputs = X_train.shape[1]
num_outputs = Y_train.shape[1]
model = Net(num_inputs, num_outputs).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_train = []

for epoch in range(2000):
    model.train()
    y = model(X_train)

    loss = criterion(y, Y_train)
    loss_train.append(loss.detach().item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"epoch {epoch}, loss: {np.mean(loss_train[-20:]):.3g}")

print(f"epoch {epoch}, loss: {np.mean(loss_train[-20:]):.3g}")

Ďalej môžeme použiť chyby uložené v `loss_train` na vykreslenie krivky učenia.



In [ ]:
plt.plot(loss_train)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid(ls='--')

Keď je tréning hotový, urobme aj štandardné vyhodnotenie na tréningových dátach. Mali by sme vidieť, že výsledky sú vcelku dobré a chyby sú v rámci škály dát zanedbateľné.



In [ ]:
Y_train_cpu = Y_train.cpu()

model.eval()
with torch.no_grad():
    y_train_cpu = model(X_train).cpu()

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_train_cpu, y_train_cpu)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_train_cpu, y_train_cpu)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_train_cpu, y_train_cpu, Y_fit_scaling=Y_train_cpu)

#### Testovanie modelu na validačných dátach

Dobre, výsledky nášho modelu na tréningových dátach sú vcelku uspokojivé. Ale platí aj, že nám model dobre zovšeobecňuje?

Keďže s návrhom siete sme ešte neskončili (nižšie budeme pokračovať), otestujeme si úspešnosť zatiaľ **nie na testovacích dátach**  (tie sa smú použiť až na konci), ale **pomocou validačných dát** . Testovacie dáta použijeme až na úplnom konci na overenie zovšeobecnenia.



In [ ]:
Y_valid_cpu = Y_valid.cpu()
Y_train_cpu = Y_train.cpu()

model.eval()
with torch.no_grad():
    y_valid_cpu = model(X_valid).cpu()

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_valid_cpu, y_valid_cpu)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_valid_cpu, y_valid_cpu)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_valid_cpu, y_valid_cpu, Y_fit_scaling=Y_train_cpu)

Po vyhodnotení modelu na validačných dátach by ste mali vidieť, že metriky sa ani len nepribližujú tým na tréningových dátach. To indikuje, že došlo k silnému **preučeniu** .

#### Testovanie na validačných dátach v priebehu učenia

Aby sme získali lepšiu predstavu o tom, kde sa učenie v rámci učenia objavili problémy, zaznamenajme si hodnoty chybovej funkcie na validačných dátach počas učenia rovnako, ako sme to robili s chybami na tréningových dátach a následne si oboje vykreslime.



In [ ]:
num_inputs = X_train.shape[1]
num_outputs = Y_train.shape[1]
model = Net(num_inputs, num_outputs).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

loss_train = []
loss_valid = []

for epoch in range(2000):
    model.train()
    y = model(X_train)

    loss = criterion(y, Y_train)
    loss_train.append(loss.detach().item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        y = model(X_valid)
        loss = criterion(y, Y_valid)
        loss_valid.append(loss.item())

    if epoch % 100 == 0:
        print(f"epoch {epoch}, train loss: {np.mean(loss_train[-20:]):.3g}, valid loss: {np.mean(loss_valid[-20:]):.3g}")

print(f"epoch {epoch}, train loss: {np.mean(loss_train[-20:]):.3g}, valid loss: {np.mean(loss_valid[-20:]):.3g}")

In [ ]:
plt.plot(loss_train, label="train")
plt.plot(loss_valid, label="valid")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid(ls='--')
plt.legend()

In [ ]:
Y_valid_cpu = Y_valid.cpu()
Y_train_cpu = Y_train.cpu()

model.eval()
with torch.no_grad():
    y_valid_cpu = model(X_valid).cpu()

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_valid_cpu, y_valid_cpu)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_valid_cpu, y_valid_cpu)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_valid_cpu, y_valid_cpu, Y_fit_scaling=Y_train_cpu)

Ako vidno, veci sa zrazu javia celkom inak. Chyba na validačných dátach je oveľa horšia než na testovacích dátach a pri pozornejšom pohľade možno uvidíte, že sa po chvíli dokonca začína zvyšovať – napriek tomu, že na tréningových dátach stále klesá či zostáva nízka.

### Regularizácia

Ako vyriešime problém uvedený v predchádzajúcej časti a zabránime preučeniu siete? Nuž, k preučeniu často dochádza preto, že keď je pre sieť ťažké znížiť chybu legitímnym spôsobom, začne podvádzať tak, že sa dáta učí naspamäť.

Aby sme predišli takýmto problémom, je potrebné použiť nejaké metódy regularizácie. Ide o metódy ktoré majú pomôcť predísť preučeniu. Názov "regularizácia" vychádza z toho, že chceme, aby náš model zachytával skutočné zákonitosti v dátach a nezačal sa dáta učiť naspamäť, aj vrátane šumu.

#### Získanie ďalších dát

Získanie väčšieho množstva dát je vo všeobecnosti najlepším spôsobom, ako zlepšiť zovšeobecnenie – pri dostatočnom množstve dát by metóda učenia mala byť schopná lepšie rozlíšiť zákonitosti od šumu. Rovnako by model nemal byť schopný zapamätať si všetky dáta, a teda je nútený učiť sa samotné zákonitosti.

Problém pri získavaní väčšieho množstva dát je ten, že je to vo všeobecnosti veľmi náročné a drahé. Preto bolo vyvinutých množstvo iných metód regularizácie – cieľom je získať čo najviac z dát, ktoré už máme.

#### Regularizácia v štandardnom strojovom učení

Vo väčšine metód strojového učenia sa regularizácia uskutočňuje tak, že sa nejakým spôsobom zníži kapacita modelu – napr. zmenšením jeho veľkosti (stupeň polynómu, veľkosť rozhodovacieho stromu, ...).

To pomáha, pretože model už nie je schopný zapamätať si tréningovú množinu a musí skutočne hľadať zákonitosti v dátach. V umelých neurónových sieťach sa to dá dosiahnuť znížením počtu a veľkosti vrstiev.

#### Skoré ukončenie učenia

Ďalším spôsobom, ako znížiť kapacitu neurálneho modelu, je použiť techniku ​​známu ako skoré ukončenie učenia. Ako sme videli už vyššie, jedna vec, ktorá sa zvyčajne stáva v priebehu tréningu, je, že aj keď chyba na tréningových dátach neustále klesá, strata na validačných dátach (ak sa používajú) prestane klesať alebo dokonca začne rásť.

Myšlienkou skorého ukončenia učenia je jednoducho prestať trénovať v tomto bode a obnoviť váhy siete do bodu, kedy bola validačná chyba na minime. Ďalšou výhodou tohto prístupu je, že prináša úsporu výpočtov.

#### Regularizácia v hlbokom učení

Oblasť hlbokého učenia je tak trochu výnimkou, pretože regularizácia sa zvyčajne nevykonáva obmedzením veľkosti modelu. Odborníci na hlboké učenie využívajú skôr:

* Špeciálne vrstvy;
* Dômyselné architektúry, ktoré zanášajú do modelu lepšie indukčné preferencie (t. j. prispôsobujú ho druhu riešenia, o ktorom je možné predpokladať, že bude dobre zovšeobecňovať);
* Zväčšovanie dátovej množiny (napr. generovanie nových náhodných variantov existujúcich vzoriek);
* Transfer učenie (t. j. predtrénovanie na väčšom množstve údajov);
*...
#### Čo budeme používať v tomto notebooku

V tomto konkrétnom notebooku nebudeme veci zbytočne komplikovať. Použijeme iba dve jednoduché metódy regularizácie:

* **Skoré ukončenie učenia** ;
* **Dropout** ;
Za zmienku stojí, že keďže neurónová sieť, ktorú tu používame, je **plytká**  a množina údajov je malá, zmenšenie **neurónovej siete**  môže byť v skutočnosti tiež dobrým spôsobom, ako regularizovať – aj keď v hlbokej siete trénovanej na miliónoch vzoriek by ste to isté nerobili.

Znova si všimnite, že počas celého procesu vývoja modelu **používame trénovaciu a validačnú množinu** , ale **nie**  testovaciu množinu. Testovaciu množinu nechávame bokom, aby sme mohli vyhodnotiť úplne finálnu verziu nášho modelu.

### Skoré ukončenie učenia

Začnime teda skorým ukončením učenia. Keďže chyby môžu byť trochu zašumené, skoré ukončenie má zvyčajne hyperparameter "patience" (trpezlivosť) – tento udáva, koľko krokov sa bude čakať potom, ako sa chyba prestane znižovať, kým sa tréning skutočne zastaví.



In [ ]:
early_stopping = EarlyStopping(checkpoint_path="output/best_model.pt")

num_inputs = X_train.shape[1]
num_outputs = Y_train.shape[1]
model = Net(num_inputs, num_outputs).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

loss_train = []
loss_valid = []

for epoch in range(2000):
    model.train()
    y = model(X_train)

    loss = criterion(y, Y_train)
    loss_train.append(loss.detach().item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        y = model(X_valid)
        loss = criterion(y, Y_valid)
        loss_valid.append(loss.item())
        if early_stopping(loss_valid[-1], model):
            print(f"Stopping the training early because the validation loss has not improved in the last {early_stopping.patience} epochs")
            break

    if epoch % 100 == 0:
        print(f"epoch {epoch}, train loss: {np.mean(loss_train[-20:]):.3g}, valid loss: {np.mean(loss_valid[-20:]):.3g}")

In [ ]:
plt.plot(loss_train, label="train")
plt.plot(loss_valid, label="valid")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid(ls='--')
plt.legend()

Po dokončení učenia načítame najlepší model späť z checkpointového súboru a spustíme vyhodnotenie. Výsledky už môžu byť o niečo lepšie – ale je tiež možné, že bude potrebná silnejšia regularizácia.



In [ ]:
model.load_state_dict(torch.load("output/best_model.pt"));

In [ ]:
Y_valid_cpu = Y_valid.cpu()
Y_train_cpu = Y_train.cpu()

model.eval()
with torch.no_grad():
    y_valid_cpu = model(X_valid).cpu()

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_valid_cpu, y_valid_cpu)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_valid_cpu, y_valid_cpu)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_valid_cpu, y_valid_cpu, Y_fit_scaling=Y_train_cpu)

### Metóda Dropout

The other kind of regularization that we are going to explore in this notebook is called **dropout** . This method will turn off a portion of neurons in a layer randomly (during training, not at evaluation time). In PyTorch this can be done by placing `nn.Dropout` after a layer. Dropout tends to make the network more robust, improving generalization.

The portion of neurons to be turned off is a hyperparameter. If we wanted to use 0.3, we could add dropout in the following way:

Ďalší druh regularizácie, na ktorý sa pozrieme v tomto notebook-u, sa nazýva **dropout** . Táto metóda náhodne vypne časť neurónov vo vrstve (počas tréningu, nie v režime hodnotenia). V PyTorch-i to možno urobiť umiestnením operácie `nn.Dropout` za príslušnú vrstvu. Dropout má tendenciu robiť sieť robustnejšou, čím sa zlepšuje jej schopnosť zovšeobecňovať.

Podiel neurónov, ktoré sa majú vypnúť, je hyperparameter. Ak by sme chceli použiť 0.3, mohli by sme pridať dropout nasledujúcim spôsobom:

```
class Net(nn.Module):
    def __init__(self):

        ...

        self.dropout = nn.Dropout(0.3)

        ...

    def forward(self, x):

        ...

        y = torch.relu(y)
        y = self.dropout(y)

        ...
```
Dropout sa typicky nezaraďuje za výstupnú vrstvu (keďže sa z nej priamo odoberajú výstupy, nulovanie prvkov by spôsobilo chybu, ktorej by nevedela predísť ani akokoľvek robustná sieť).

#### Dropout a kapacita modelu

Ak sa použije agresívnejšia forma regularizácie, kapacitu modelu to môže znížiť podstatne. Je teda napríklad možné, že model silno využívajúci dropout bude potrebovať vrstvy s o niečo väčším počtom neurónov než model bez `Dropout` vrstiev.

Netriviálna je aj interakcia medzi rôznymi metódami regularizácie: napríklad pri použití metódy dropout sa dá očakávať, že bude vo výsledkoch na validačnej množine väčší rozptyl (na zmeny váh vplývajú ďalšie stochastické faktory); preto môže byť v prípade s kombináciou so skorým ukončením učenia potrebné použiť podstatne vyššiu hodnotu `patience`.

---
### Úloha 3

**Skúste do siete vložiť niekoľko `Dropout` vrstiev. Napríklad jednu `Dropout` vrstvu za každú `relu` vrstvu.** 

**Efektivitu regularizácie testujte počas ladenia parametrov len pomocou validačnej dátovej množiny. Testovacia dátová množina sa použije až nakoniec – len jeden raz!** 

---


In [ ]:
class DropoutNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()


        # -----
        


Pokúsme sa zopakovať tréning znovu s použitím našej novej siete. Aby sme si postup zjednodušili a nemuseli ladiť parameter `patience` (prinízka hodnota by mohla naše výsledky zhoršiť), v tomto behu nebudeme používať skoré ukončenie učenia.



In [ ]:
num_inputs = X_train.shape[1]
num_outputs = Y_train.shape[1]
model = DropoutNet(num_inputs, num_outputs).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

loss_train = []
loss_valid = []

for epoch in range(2000):
    model.train()
    y = model(X_train)

    loss = criterion(y, Y_train)
    loss_train.append(loss.detach().item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        y = model(X_valid)
        loss = criterion(y, Y_valid)
        loss_valid.append(loss.item())

    if epoch % 100 == 0:
        print(f"epoch {epoch}, train loss: {np.mean(loss_train[-20:]):.3g}, valid loss: {np.mean(loss_valid[-20:]):.3g}")

In [ ]:
plt.plot(loss_train, label="train")
plt.plot(loss_valid, label="valid")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid(ls='--')
plt.legend()

Všimnite si, že sa naša validačná chyba už nezvyšuje. Všimnite si tiež, že chyby obsahujú viac šumu – je to, samozrejme, kvôli šumu, ktorý zavádza dropout.

Chyby na tréningových a na validačných dátach by sa teraz nemali líšiť až tak výrazne.



In [ ]:
Y_train_cpu = Y_train.cpu()

model.eval()
with torch.no_grad():
    y_train_cpu = model(X_train).cpu()

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_train_cpu, y_train_cpu)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_train_cpu, y_train_cpu)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_train_cpu, y_train_cpu, Y_fit_scaling=Y_train_cpu)

In [ ]:
Y_valid_cpu = Y_valid.cpu()
Y_train_cpu = Y_train.cpu()

model.eval()
with torch.no_grad():
    y_valid_cpu = model(X_valid).cpu()

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_valid_cpu, y_valid_cpu)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_valid_cpu, y_valid_cpu)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_valid_cpu, y_valid_cpu, Y_fit_scaling=Y_train_cpu)

### Výsledky na testovacej množine

Potom, ako sme sa dopracovali k finálnej verzii modelu, otestujeme jeho zovšeobecnenie nakoniec aj na testovacích dátach. 

Keďže sme v našom finálnom modeli nepoužili skoré ukončenie učenia, mohli by sme teraz pred testovaním zopakovať tréning ešte raz na tréningových + validačných dátach. To by mohlo ešte trochu zlepšiť výsledky nášho záverečného testu – ak chcete, môžete pridať potrebný kód.



In [ ]:
Y_test_cpu = Y_test.cpu()
Y_train_cpu = Y_train.cpu()

model.eval()
with torch.no_grad():
    y_test_cpu = model(X_test).cpu()

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_test_cpu, y_test_cpu)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_test_cpu, y_test_cpu)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_test_cpu, y_test_cpu, Y_fit_scaling=Y_train_cpu)

### Regresia pomocou rozhodovacích stromov s gradientným boostingom

Aby sme ukázali, že neurónové siete nemajú na štruktúrovaných dátach podstatnú výhodu a lepšie výsledky sa väčšinou dajú dosiahnuť inými metódami, porovnáme výsledky aj s metódou XGBoost založenou na komisii rozhodovacích stromov a gradientnom boosting-u. Je dobrá šanca, že výsledok bude lepší než sa nám podarilo dosiahnuť pomocou neurónovej siete: a učenie bude trvať podstatne kratší čas. Skutočné výhody neurónových sietí typicky vidno až pri náročnejších neštruktúrovaných dátach ako sú obraz, zvuk a pod.



In [ ]:
from xgboost import XGBRegressor
X_train_np = X_train.cpu().numpy()
Y_train_np = Y_train.cpu().numpy()
X_test_np = X_test.cpu().numpy()
Y_test_np = Y_test.cpu().numpy()

model = XGBRegressor()
model.fit(X_train_np, Y_train_np);

In [ ]:
y_test = model.predict(X_test_np)

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_test_np, y_test)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_test_np, y_test)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_test_np, y_test, Y_fit_scaling=Y_train_np)